In [1]:
!pip install datasets duckdb sentence-transformers model2vec vicinity gradio gradio-client -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 68.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import duckdb
import gradio as gr
import numpy as np
import pandas as pd

from datasets import load_dataset
from gradio_client import Client
from sentence_transformers import SentenceTransformer
from sentence_transformers.models import StaticEmbedding
from vicinity import Vicinity, Backend, Metric


In [10]:
dataset = load_dataset("pythainlp/thaigov-corpus")
dataset["train"]


README.md:   0%|          | 0.00/4.49k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/85.8M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/87.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/28555 [00:00<?, ? examples/s]

Dataset({
    features: ['title', 'context', 'raw', 'url'],
    num_rows: 28555
})

In [12]:
pip install -U FlagEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.0/859.0 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.2/119.2 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.5 MB/s eta 0:00:00
  Created wheel for FlagEmbedding: filename=FlagEmbedding-1.3.4-py3-none-any.whl size=232500 sha256=0e9f66db3829d7ffb40ba149816e1c2bf3178dcb176422a107c8ba9809ca0e1f
  Stored in directory: /root/.cache/pip/wheels/57/69/8b/bb209413e16cb21065716300c291e75ca5949c878283836c4d
  Created wheel for warc3-wet-clueweb09: filename=warc3_wet_clueweb09-0.2.5-py3-none-any.whl size=18919 sha256=35aa87a3435595adfaa0855a3e36bc999a7f383a492e8298ea5cac3012f675b9
  Stored in directory: /root/.

In [16]:
from FlagEmbedding import BGEM3FlagModel


model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)

def create_embeddings(batch):
    """Create embeddings for a batch of text chunks."""
    encoding_results = model.encode(
        batch["context"],
        return_dense=True,
        return_sparse=False,
        return_colbert_vecs=False
    )

    batch["embedding"] = encoding_results['dense_vecs']
    return batch

embeddings_dataset = dataset.map(create_embeddings, batched=True)


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

Map:   0%|          | 0/28555 [00:00<?, ? examples/s]


pre tokenize: 100%|██████████| 4/4 [00:02<00:00,  1.54it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.

Inference Embeddings: 100%|██████████| 4/4 [00:25<00:00,  6.28s/it]

pre tokenize: 100%|██████████| 4/4 [00:03<00:00,  1.28it/s]

Inference Embeddings: 100%|██████████| 4/4 [00:22<00:00,  5.70s/it]

pre tokenize: 100%|██████████| 4/4 [00:03<00:00,  1.06it/s]

Inference Embeddings: 100%|██████████| 4/4 [00:22<00:00,  5.59s/it]

pre tokenize: 100%|██████████| 4/4 [00:03<00:00,  1.23it/s]

Inference Embeddings: 100%|██████████| 4/4 [00:23<00:00,  5.81s/it]

pre tokenize: 100%|██████████| 4/4 [00:02<00:00,  1.44it/s]

Inference Embeddings: 100%|██████████| 4/4 [00:23<00:00,  5.77s/it]

pre tokenize: 100%|██████████| 4/4 [00:02<00:00,  1.35it/s]

Inference Embeddings: 100%|██████████| 4/4 [00:22<00:00,

In [17]:
embeddings_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'raw', 'url', 'embedding'],
        num_rows: 28555
    })
})

In [23]:
import duckdb
from FlagEmbedding import BGEM3FlagModel
import pandas as pd
import numpy as np

# Initialize the BGE-M3 model
model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)
EMBEDDING_DIMENSION = 1024

def _setup_vss():
    return """
        INSTALL vss;
        LOAD vss;
        """

def _drop_table(table_name):
    return f"""
        DROP TABLE IF EXISTS {table_name};
        """

def _load_raw_data(dataset_name, raw_table_name):
    """Load the raw data from HF dataset without trying to access embeddings"""
    return f"""
        CREATE TABLE {raw_table_name} AS
        SELECT *
        FROM 'hf://datasets/{dataset_name}/**/*.parquet';
        """

def _inspect_table(table_name):
    """Utility to inspect table structure"""
    return f"DESCRIBE {table_name};"

def create_search_table():
    """Create search table with original data and embeddings"""
    duckdb.sql(_setup_vss())

    raw_table = "thaigov_raw"
    duckdb.sql(_drop_table(raw_table))
    duckdb.sql(_load_raw_data("pythainlp/thaigov-corpus", raw_table))

    print("Dataset structure:")
    structure = duckdb.sql(_inspect_table(raw_table)).fetchall()
    for col in structure:
        print(f"- {col[0]}: {col[1]}")

    text_query = f"SELECT rowid, title, context FROM {raw_table} "
    raw_data = duckdb.sql(text_query).df()

    contexts = raw_data['context'].tolist()

    print(f"Generating embeddings for {len(contexts)} documents...")
    batch_size = 32*12
    all_embeddings = []

    for i in range(0, len(contexts), batch_size):
        batch = contexts[i:i+batch_size]
        results = model.encode(batch, return_dense=True)
        all_embeddings.extend(results['dense_vecs'])
        if i % 320 == 0 and i > 0:
            print(f"Processed {i} documents...")

    print("Embedding generation complete.")

    embedding_strings = []
    for embedding in all_embeddings:
        embedding_str = '[' + ','.join(map(str, embedding)) + ']'
        embedding_strings.append(embedding_str)

    embeddings_df = pd.DataFrame({
        'rowid': raw_data['rowid'],
        'embedding': embedding_strings
    })

    embed_table = "temp_embeddings"
    duckdb.sql(_drop_table(embed_table))
    duckdb.sql(f"CREATE TABLE {embed_table} AS SELECT * FROM embeddings_df")

    final_table = "thaigov_search"
    duckdb.sql(_drop_table(final_table))

    join_query = f"""
        CREATE TABLE {final_table} AS
        SELECT
            r.title,
            r.context,
            r.url,
            e.embedding::float[{EMBEDDING_DIMENSION}] as embedding_float
        FROM
            {raw_table} r
        JOIN
            {embed_table} e ON r.rowid = e.rowid;
    """
    duckdb.sql(join_query)

    index_query = f"""
        CREATE INDEX thaigov_hnsw_index ON {final_table}
        USING HNSW (embedding_float) WITH (metric = 'cosine');
    """
    duckdb.sql(index_query)

    print(f"Search table and index created: {final_table}")
    return final_table

def similarity_search(query_text, table_name="thaigov_search", top_k=5):
    """Perform similarity search using the vector index"""
    result = model.encode([query_text], return_dense=True)
    query_embedding = result['dense_vecs'][0]

    embedding_str = '[' + ','.join(map(str, query_embedding)) + ']'

    search_query = f"""
        SELECT
            title,
            context,
            url,
            cosine_similarity(embedding_float, {embedding_str}::float[{EMBEDDING_DIMENSION}]) as similarity
        FROM {table_name}
        ORDER BY similarity DESC
        LIMIT {top_k};
    """

    return duckdb.sql(search_query).fetchall()

search_table = create_search_table()



Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Dataset structure:
- title: VARCHAR
- context: VARCHAR
- raw: VARCHAR
- url: VARCHAR
Generating embeddings for 28555 documents...


pre tokenize: 100%|██████████| 1/1 [00:00<00:00,  1.51it/s]
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Inference Embeddings: 100%|██████████| 1/1 [00:09<00:00,  9.67s/it]


Processed 1280 documents...


...

Inference Embeddings: 100%|██████████| 1/1 [00:09<00:00,  9.78s/it]


Processed 28160 documents...
Embedding generation complete.


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Search table and index created: thaigov_search


In [27]:
from FlagEmbedding import BGEM3FlagModel
import duckdb

# Make sure the model is initialized
model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)
EMBEDDING_DIMENSION = 1024  # BGE-M3 dimension

# Make sure VSS extension is installed and loaded
duckdb.sql("INSTALL vss")
duckdb.sql("LOAD vss")

# Let's check what functions are available in the VSS extension
try:
    functions = duckdb.sql("SELECT function_name FROM duckdb_functions() WHERE function_name LIKE '%vector%' OR function_name LIKE '%distance%' OR function_name LIKE '%cosine%' OR function_name LIKE '%similarity%'").fetchall()
    print("Available vector-related functions:")
    for func in functions:
        print(f"- {func[0]}")
except:
    print("Could not fetch function list")

def similarity_search_with_duckdb_index(
    query: str,
    k: int = 5,
    table_name: str = "thaigov_search",
    embedding_column: str = "embedding_float"
):
    result = model.encode([query], return_dense=True)
    embedding = result['dense_vecs'][0]

    embedding_list = embedding.tolist()

    try:
        print("Trying cosine_distance...")
        df = duckdb.sql(
            query=f"""
            SELECT
                title,
                context,
                url,
                1 - cosine_distance({embedding_column}, {embedding_list}::FLOAT[{EMBEDDING_DIMENSION}]) as similarity
            FROM {table_name}
            ORDER BY similarity DESC
            LIMIT {k};
            """
        ).to_df()
        return df
    except Exception as e:
        print(f"cosine_distance failed: {e}")

    # Try alternative 2: array_cosine_distance
    try:
        print("Trying array_cosine_distance...")
        df = duckdb.sql(
            query=f"""
            SELECT
                title,
                context,
                url,
                1 - array_cosine_distance({embedding_column}, {embedding_list}::FLOAT[{EMBEDDING_DIMENSION}]) as similarity
            FROM {table_name}
            ORDER BY similarity DESC
            LIMIT {k};
            """
        ).to_df()
        return df
    except Exception as e:
        print(f"array_cosine_distance failed: {e}")

    try:
        print("Trying hamming distance...")
        df = duckdb.sql(
            query=f"""
            SELECT
                title,
                context,
                url,
                1 - hamming({embedding_column}, {embedding_list}::FLOAT[{EMBEDDING_DIMENSION}]) as similarity
            FROM {table_name}
            ORDER BY similarity DESC
            LIMIT {k};
            """
        ).to_df()
        return df
    except Exception as e:
        print(f"hamming failed: {e}")

    try:
        print("Trying euclidean distance...")
        df = duckdb.sql(
            query=f"""
            SELECT
                title,
                context,
                url,
                1 / (1 + euclidean({embedding_column}, {embedding_list}::FLOAT[{EMBEDDING_DIMENSION}])) as similarity
            FROM {table_name}
            ORDER BY similarity DESC
            LIMIT {k};
            """
        ).to_df()
        return df
    except Exception as e:
        print(f"euclidean failed: {e}")

    try:
        print("Trying direct HNSW search...")
        df = duckdb.sql(
            query=f"""
            SELECT
                title,
                context,
                url,
                hnsw_search({embedding_column}, {embedding_list}::FLOAT[{EMBEDDING_DIMENSION}], {k}) as similarity
            FROM {table_name}
            LIMIT {k};
            """
        ).to_df()
        return df
    except Exception as e:
        print(f"HNSW search failed: {e}")

    raise Exception("All similarity search methods failed")

try:
    results = similarity_search_with_duckdb_index(
        "ไวรัสโควิด-19 สามารถแพร่กระจายเชื้อได้อย่างไร รัฐบาลมีมาตรการแก้ไขอย่างไร?",
        k=5
    )

    for i, row in results.iterrows():
        print(f"Result {i+1} (Similarity: {row['similarity']:.4f})")
        print(f"Title: {row['title']}")
        print(f"URL: {row['url']}")
        print(f"Context excerpt: {row['context'][:200]}...")
        print("-" * 80)
except Exception as e:
    print(f"Search failed: {e}")

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

Available vector-related functions:
- test_vector_types
- vector_type
- list_distance
- list_distance
- list_cosine_distance
- list_cosine_distance
- array_cosine_distance
- array_cosine_distance
- array_distance
- array_distance
- list_cosine_similarity
- list_cosine_similarity
- array_cosine_similarity
- array_cosine_similarity
- jaro_similarity
- jaro_similarity
- jaro_winkler_similarity
- jaro_winkler_similarity


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trying cosine_distance...
cosine_distance failed: Catalog Error: Scalar Function with name cosine_distance does not exist!
Did you mean "cos"?
Trying array_cosine_distance...
Result 1 (Similarity: 0.6564)
Title: รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-มท.1 ย้ำรัฐบาลดำเนินมาตรการป้องกันโควิด-19 เป็นขั้นตอนอย่างเป็นระบบ ประเมินสถานการณ์ทุกระยะ รวมทั้งได้รับความร่วมมือจากประชาชนและทุกภาคส่วน จึงยับยั้งโควิดได้ [กระทรวงมหาดไทย]
URL: https://www.thaigov.go.th/news/contents/details/31723
Context excerpt: วันเสาร์ที่ 30 พฤษภาคม 2563
มท.1 ย้ํารัฐบาลดําเนินมาตรการป้องกันโควิด-19 เป็นขั้นตอนอย่างเป็นระบบ ประเมินสถานการณ์ทุกระยะ รวมทั้งได้รับความร่วมมือจากประชาชนและทุกภาคส่วน จึงยับยั้งโควิดได้ [กระทรวงมหา...
--------------------------------------------------------------------------------
Result 2 (Similarity: 0.6502)
Title: รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-มท.1 ย้ำรัฐบาลดำเนินมาตรการป้องกันโควิด-19 เป็นขั้นตอนอย่างเป็นระบบ ประเมินสถานการณ์ทุกระยะ
URL: https://www.thaigov.go.th/news/contents/details/31741
Context

In [29]:
from FlagEmbedding import BGEM3FlagModel
import duckdb
import pandas as pd
from IPython.display import display, HTML

# Make sure the model is initialized
model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)
EMBEDDING_DIMENSION = 1024  # BGE-M3 dimension

# Make sure VSS extension is installed and loaded
duckdb.sql("INSTALL vss")
duckdb.sql("LOAD vss")

def similarity_search_with_duckdb_index(
    query: str,
    k: int = 5,
    table_name: str = "thaigov_search",
    embedding_column: str = "embedding_float",
    return_formatted: bool = True
):
    """
    Perform similarity search using DuckDB vector index.

    Args:
        query: The search query text
        k: Number of results to return
        table_name: Name of the table with embeddings
        embedding_column: Name of the embedding column
        return_formatted: If True, format results as HTML table

    Returns:
        DataFrame with search results or HTML table if return_formatted=True
    """
    # Get the embedding from BGE-M3 model
    result = model.encode([query], return_dense=True)
    embedding = result['dense_vecs'][0]  # Extract the embedding array

    # Convert to a list for DuckDB
    embedding_list = embedding.tolist()

    # Try different distance functions that might be available
    possible_functions = [
        # Format: (query_template, needs_inversion)
        (f"1 - cosine_distance({embedding_column}, {{embedding}}::FLOAT[{EMBEDDING_DIMENSION}])", False),
        (f"1 - array_cosine_distance({embedding_column}, {{embedding}}::FLOAT[{EMBEDDING_DIMENSION}])", False),
        (f"1 / (1 + euclidean({embedding_column}, {{embedding}}::FLOAT[{EMBEDDING_DIMENSION}]))", False),
    ]

    df = None
    for query_template, needs_inversion in possible_functions:
        try:
            similarity_expr = query_template.format(embedding=embedding_list)
            order_direction = "DESC" if not needs_inversion else "ASC"

            query_str = f"""
            SELECT
                title,
                context,
                url,
                {similarity_expr} as similarity
            FROM {table_name}
            ORDER BY similarity {order_direction}
            LIMIT {k};
            """

            df = duckdb.sql(query_str).to_df()
            break  # If successful, exit the loop
        except Exception as e:
            print(f"Function failed: {e}")
            continue

    if df is None:
        raise Exception("All similarity search methods failed")

    if return_formatted:
        formatted_df = df.copy()

        formatted_df['context'] = formatted_df['context'].apply(lambda x: x[:250] + "..." if len(x) > 250 else x)

        formatted_df['similarity'] = formatted_df['similarity'].apply(lambda x: f"{x:.2%}")

        formatted_df.columns = ['Title', 'Content Preview', 'URL', 'Similarity']

        styled_table = formatted_df.style.set_properties(**{
            'text-align': 'left',
            'white-space': 'pre-wrap',
            'border': '1px solid #cccccc',
            'padding': '10px'
        }).set_table_styles([
            {'selector': 'th', 'props': [('background-color', '#f2f2f2'),
                                         ('color', '#333'),
                                         ('font-weight', 'bold'),
                                         ('text-align', 'center'),
                                         ('padding', '10px')]},
            {'selector': 'tr:nth-of-type(odd)', 'props': [('background-color', '#f9f9f9')]},
            {'selector': 'td', 'props': [('padding', '10px')]}
        ]).hide(axis="index")

        return styled_table

    return df

query = "ไวรัสโควิด-19 สามารถแพร่กระจายเชื้อได้อย่างไร รัฐบาลมีมาตรการแก้ไขอย่างไร?"
results = similarity_search_with_duckdb_index(query, k=5)

display(results)

raw_df = similarity_search_with_duckdb_index(query, k=5, return_formatted=False)

Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Function failed: Catalog Error: Scalar Function with name cosine_distance does not exist!
Did you mean "cos"?


Title,Content Preview,URL,Similarity
รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-มท.1 ย้ำรัฐบาลดำเนินมาตรการป้องกันโควิด-19 เป็นขั้นตอนอย่างเป็นระบบ ประเมินสถานการณ์ทุกระยะ รวมทั้งได้รับความร่วมมือจากประชาชนและทุกภาคส่วน จึงยับยั้งโควิดได้ [กระทรวงมหาดไทย],วันเสาร์ที่ 30 พฤษภาคม 2563 มท.1 ย้ํารัฐบาลดําเนินมาตรการป้องกันโควิด-19 เป็นขั้นตอนอย่างเป็นระบบ ประเมินสถานการณ์ทุกระยะ รวมทั้งได้รับความร่วมมือจากประชาชนและทุกภาคส่วน จึงยับยั้งโควิดได้ [กระทรวงมหาดไทย] มท.1 ย้ํารัฐบาลดําเนินมาตรการป้องกันโควิด-19...,https://www.thaigov.go.th/news/contents/details/31723,65.64%
รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-มท.1 ย้ำรัฐบาลดำเนินมาตรการป้องกันโควิด-19 เป็นขั้นตอนอย่างเป็นระบบ ประเมินสถานการณ์ทุกระยะ,วันอาทิตย์ที่ 31 พฤษภาคม 2563 มท.1 ย้ํารัฐบาลดําเนินมาตรการป้องกันโควิด-19 เป็นขั้นตอนอย่างเป็นระบบ ประเมินสถานการณ์ทุกระยะ มท.1 ย้ํารัฐบาลดําเนินมาตรการป้องกันโควิด-19 เป็นขั้นตอนอย่างเป็นระบบ ประเมินสถานการณ์ทุกระยะ รวมทั้งได้รับความร่วมมือจากประชา...,https://www.thaigov.go.th/news/contents/details/31741,65.02%
รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-ไวรัสโควิด-19 สามารถแพร่กระจายเชื้อได้อย่างไร ??,วันอาทิตย์ที่ 15 มีนาคม 2563 ไวรัสโควิด-19 สามารถแพร่กระจายเชื้อได้อย่างไร ?? ไวรัสโควิด-19 สามารถแพร่กระจายเชื้อได้อย่างไร ?? Q: ไวรัสโควิด-19 สามารถแพร่กระจายเชื้อได้อย่างไร ?? A: ส่วนใหญ่แพร่กระจายผ่านการสัมผัสกับผู้ติดเชื้อ ผ่านทางละอองเสมหะจากกา...,https://www.thaigov.go.th/news/contents/details/27331,63.03%
รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-กระทรวงการคลังร่วมใจสู้ภัย COVID-19 [กระทรวงการคลัง],วันเสาร์ที่ 14 มีนาคม 2563 กระทรวงการคลังร่วมใจสู้ภัย COVID-19 [กระทรวงการคลัง] พลเอกประยุทธ์ จันทร์โอชา นายกรัฐมนตรี ได้มีแนวนโยบายพร้อมกําชับให้ทุกภาคส่วนร่วมกันป้องกันและแก้ไขปัญหาการแพร่ระบาดของเชื้อไวรัสโคโรนา 2019 หรือโควิด 19 ด้วยพลเอกประยุทธ์...,https://www.thaigov.go.th/news/contents/details/27298,62.24%
รัฐบาลไทย-ข่าวทำเนียบรัฐบาล-รัฐบาลวางแผนกลยุทธ์ระยะยาวสู้โควิด-19 “หยุดเชื้อ อยู่บ้าน เพื่อชาติ” มั่นใจสินค้ามีเพียงพอ,วันศุกร์ที่ 20 มีนาคม 2563 รัฐบาลวางแผนกลยุทธ์ระยะยาวสู้โควิด-19 “หยุดเชื้อ อยู่บ้าน เพื่อชาติ” มั่นใจสินค้ามีเพียงพอ รัฐบาลวางแผนกลยุทธ์ระยะยาวสู้โควิด-19 “หยุดเชื้อ อยู่บ้าน เพื่อชาติ” มั่นใจสินค้ามีเพียงพอ วันนี้ (20มี.ค.2563)เวลา14.00น. ณ ศูนย์แถ...,https://www.thaigov.go.th/news/contents/details/27604,62.13%


Function failed: Catalog Error: Scalar Function with name cosine_distance does not exist!
Did you mean "cos"?
